<h1 align="center"> Assignment 4: Clustering</h1><br>

In [238]:
import numpy as np
import glob
import pandas as pd

import h5py as h5
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, SelectKBest
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR, LinearSVR
from sklearn import preprocessing
import calendar as cal
import seaborn as sns
import datetime as dt
%matplotlib inline

In [239]:
from pandas import HDFStore
store = HDFStore('/Users/mgalarny/Desktop/MachineLearningProject/yogurt.h5')
store.keys()

['/delivery_stores', '/demos', '/drug', '/groc', '/panel']

<strong><p style="font-size:120%;">Loading the Data </strong> My group has merged all the csv files into one H5 File

In [240]:
demo = pd.read_hdf('/Users/mgalarny/Desktop/MachineLearningProject/yogurt.h5','demos')
demo.columns

Index([u'ALL_TVS', u'Age Group Applied to Female HH',
       u'Age Group Applied to Male HH', u'CABL_TVS', u'COUNTY',
       u'Children Group Code', u'Combined Pre-Tax Income of HH', u'EXT_FACT',
       u'Education Level Reached by Female HH',
       u'Education Level Reached by Male HH', u'FIPSCODE', u'Family Size',
       u'Female Working Hour Code', u'HH_AGE', u'HH_EDU', u'HH_OCC',
       u'HH_RACE', u'IRI Geography Number', u'Language',
       u'Male Working Hour Code', u'Marital Status', u'Number of Cats',
       u'Number of Dogs', u'Occupation Code of Female HH',
       u'Occupation Code of Male HH', u'Panelist ID', u'RACE3',
       u'Type of Residential Possession', u'Year', u'ZIPCODE', u'filename',
       u'yearID'],
      dtype='object')

<strong><p style="font-size:120%;">Inspecting the Data</strong> Looking for columns with NaNs in them

In [241]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74592 entries, 0 to 4606
Data columns (total 32 columns):
ALL_TVS                                 57426 non-null float64
Age Group Applied to Female HH          74592 non-null int64
Age Group Applied to Male HH            74592 non-null int64
CABL_TVS                                54537 non-null float64
COUNTY                                  46722 non-null object
Children Group Code                     74592 non-null int64
Combined Pre-Tax Income of HH           74581 non-null float64
EXT_FACT                                46722 non-null float64
Education Level Reached by Female HH    74592 non-null int64
Education Level Reached by Male HH      74592 non-null int64
FIPSCODE                                46722 non-null float64
Family Size                             74592 non-null int64
Female Working Hour Code                74592 non-null int64
HH_AGE                                  46627 non-null float64
HH_EDU                   

<strong><p style="font-size:120%;">Look at Documentation </strong> Specifically pg 26-27 of the Research and Development data set and field description

In [242]:
demo.columns

Index([u'ALL_TVS', u'Age Group Applied to Female HH',
       u'Age Group Applied to Male HH', u'CABL_TVS', u'COUNTY',
       u'Children Group Code', u'Combined Pre-Tax Income of HH', u'EXT_FACT',
       u'Education Level Reached by Female HH',
       u'Education Level Reached by Male HH', u'FIPSCODE', u'Family Size',
       u'Female Working Hour Code', u'HH_AGE', u'HH_EDU', u'HH_OCC',
       u'HH_RACE', u'IRI Geography Number', u'Language',
       u'Male Working Hour Code', u'Marital Status', u'Number of Cats',
       u'Number of Dogs', u'Occupation Code of Female HH',
       u'Occupation Code of Male HH', u'Panelist ID', u'RACE3',
       u'Type of Residential Possession', u'Year', u'ZIPCODE', u'filename',
       u'yearID'],
      dtype='object')

<strong><p style="font-size:120%;">Drop Non Useful Columns </strong>

In [243]:
try: 
    demo = demo.drop(['EXT_FACT', 'ZIPCODE', 'ALL_TVS', \
                      'CABL_TVS', 'FIPSCODE', 'IRI Geography Number', \
                      'ZIPCODE', 'filename', 'RACE3', \
                      'Year', 'Panelist ID', 'COUNTY', \
                      'yearID', 'HH_AGE', 'HH_EDU', \
                      'HH_RACE', 'HH_OCC', 'Language', \
                      'Occupation Code of Female HH', \
                      'Occupation Code of Male HH'], axis=1)
except: 
    pass

demo.tail()

,Age Group Applied to Female HH,Age Group Applied to Male HH,Children Group Code,Combined Pre-Tax Income of HH,Education Level Reached by Female HH,Education Level Reached by Male HH,Family Size,Female Working Hour Code,Male Working Hour Code,Marital Status,Number of Cats,Number of Dogs,Type of Residential Possession
4602,4,7,8,1.0,3,0,1,3,4,3,0,0,2
4603,6,6,8,9.0,4,4,2,3,3,1,0,0,1
4604,6,6,8,5.0,4,5,2,3,3,1,0,0,1
4605,6,6,8,7.0,3,3,2,3,3,1,0,0,1
4606,4,7,3,8.0,5,0,3,2,4,3,0,0,2


In [244]:
drop_reasons = {'Column': ['EXT_FACT', 'ZIPCODE', 'ALL_TVS', \
            'CABL_TVS', 'FIPSCODE', 'IRI Geography Number', \
            'Language', 'ZIPCODE', 'filename', \
            'RACE3', 'Year','Panelist ID', \
            'COUNTY', 'yearID', 'HH_AGE', \
            'HH_EDU', 'HH_OCC', 'Language', \
            'Occupation Code of Female HH', 'Occupation Code of Male HH'], \
'Reason for Dropping': \
             ['equal market/demo weight is too strongly correlated with other variables', \
             'Too many different zipcodes', 'Not alot of documentation', 'Not interested', \
             'Each is unique', 'Geography isn\'t of interest', 'Language is too highly correlated to race (HH_RACE)', \
             'Unique', 'Not Relevant', 'Too Many NaNs', \
             'not interesting', 'unique', 'Not sure I should have dropped this', 'Already dropped year', \
             'NaNs, may add later after initial analysis', 'may add later after initial analysis', \
             'may add later after initial analysis', 'may add later after initial analysis', \
             'Already have college and salary', 'Already have college and salary']}

pd.DataFrame(data = drop_reasons)

,Column,Reason for Dropping
0,EXT_FACT,equal market/demo weight is too strongly corre...
1,ZIPCODE,Too many different zipcodes
2,ALL_TVS,Not alot of documentation
3,CABL_TVS,Not interested
4,FIPSCODE,Each is unique
5,IRI Geography Number,Geography isn't of interest
6,Language,Language is too highly correlated to race (HH_...
7,ZIPCODE,Unique
8,filename,Not Relevant
9,RACE3,Too Many NaNs


<strong><p style="font-size:120%;">Feature Engineering </strong> A good part of this was transforming the values to be more understandable for clustering later on. 

In [245]:
# Making Numbers Easier to interpret for Clustering Analysis


age_dict = {0: 'N/A', 
                  1: '18-24',
                  2: '25-34',
                  3: '35-44',
                  4: '45-54',
                  5: '55-64',
                  6: '65 +',
                  7: 'No such person'}

edu_dict = {0: 'N/A', 
            1: 'Some grade school or less',
            2: 'Completed grade school',
            3: 'Some high school',
            4: 'Graduated high school',
            5: 'Technical school',
            6: 'Some college',
            7: 'Graduated from college',
            8: 'Post graduate work',
            9: 'No such head of household'}

pre_tax_income_dict = {0: 'N/A',
                  1: '$00,000 to $ 9,999 per yr',
                  2: '$10,000 to $11,999 per yr',
                  3: '$12,000 to $14,999 per yr',
                  4: '$15,000 to $19,999 per yr',
                  5: '$20,000 to $24,999 per yr',
                  6: '$25,000 to $34,999 per yr',
                  7: '$35,000 to $44,999 per yr',
                  8: '$45,000 to $54,999 per yr',
                  9: '$55,000 to $64,999 per yr',
                  10: '$65,000 to $74,999 per yr', 
                  11: '$75,000 to $99,999 per yr', 
                  12: '$100,000 and greater per year'}

working_hour_dict = {0: 'N/A', 
                     1: 'Not employed',
                     2: 'Part time, < 35 hrs./wk.',
                     3: 'Full time, > 35 hrs./wk.',
                     4: 'Retired',
                     5: 'Homemaker',
                     6: 'Some college',
                     7: 'N/A'}

marital_status_dict = {0: 'N/A',
                       1: 'Single',
                       2: 'Married',
                       3: 'Divorced',
                       4: 'Widowed',
                       5: 'Separated'}

child_group_dict = {0: 'N/A',
                    1: 'Child in [0-5)',
                    2: 'Child in [6-11)',
                    3: 'Child in [12-17)',
                    4: 'Children in [0-5) & [6-11)',
                    5: 'Children in [0-5) & [12-17)', 
                    6: 'Children in [6-11) & [1217)',
                    7: 'Children in [0-5),[6-11) & [12-17)',
                    8: 'Family size>0 yet no children'}

residental_possession_dict = {0: 'N/A',
                              1: 'Renter',
                              2: 'Owner'}

In [246]:
def handler(dictionary,key):
    # function to deal with nans and unidenified dictionary keys
    try:
        value = dictionary[key]
    except: 
        value = dictionary[0]
    return value
    

In [247]:
demo['Age Group Applied to Female HH']= demo['Age Group Applied to Female HH'].apply(lambda x: age_dict[x] )
demo['Age Group Applied to Male HH']= demo['Age Group Applied to Male HH'].apply(lambda x: age_dict[x] )
demo['Education Level Reached by Female HH']= demo['Education Level Reached by Female HH'].apply(lambda x: handler(edu_dict,x))
demo['Education Level Reached by Male HH']= demo['Education Level Reached by Male HH'].apply(lambda x: handler(edu_dict,x))
demo['Combined Pre-Tax Income of HH']= demo['Combined Pre-Tax Income of HH'].apply(lambda x: handler(pre_tax_income_dict,x))

In [248]:
demo['Male Working Hour Code']= demo['Male Working Hour Code'].apply(lambda x: handler(working_hour_dict,x))
demo['Female Working Hour Code']= demo['Female Working Hour Code'].apply(lambda x: handler(working_hour_dict,x))
demo['Marital Status']= demo['Marital Status'].apply(lambda x: handler(marital_status_dict,x))
demo['Children Group Code']= demo['Children Group Code'].apply(lambda x: handler(child_group_dict,x))
demo['Type of Residential Possession']= demo['Type of Residential Possession'].apply(lambda x: handler(residental_possession_dict,x))

In [249]:
demo.columns

Index([u'Age Group Applied to Female HH', u'Age Group Applied to Male HH',
       u'Children Group Code', u'Combined Pre-Tax Income of HH',
       u'Education Level Reached by Female HH',
       u'Education Level Reached by Male HH', u'Family Size',
       u'Female Working Hour Code', u'Male Working Hour Code',
       u'Marital Status', u'Number of Cats', u'Number of Dogs',
       u'Type of Residential Possession'],
      dtype='object')

In [253]:
demo.head(3)

,Age Group Applied to Female HH,Age Group Applied to Male HH,Children Group Code,Combined Pre-Tax Income of HH,Education Level Reached by Female HH,Education Level Reached by Male HH,Family Size,Female Working Hour Code,Male Working Hour Code,Marital Status,Number of Cats,Number of Dogs,Type of Residential Possession
0,65 +,65 +,Family size>0 yet no children,"$12,000 to $14,999 per yr",Some high school,Completed grade school,3,Retired,Retired,Married,0,0,Owner
1,55-64,No such person,Child in [12-17),"$20,000 to $24,999 per yr",Graduated from college,No such head of household,2,"Full time, > 35 hrs./wk.",Homemaker,Single,2,0,Owner
2,65 +,65 +,Family size>0 yet no children,"$45,000 to $54,999 per yr",Some college,Technical school,2,"Full time, > 35 hrs./wk.",Retired,Married,0,1,Owner


In [260]:
prefixes = {'Age Group Applied to Female HH': 'Age Female HH ', 
            'Age Group Applied to Male HH': 'Age Male HH ', 
            'Children Group Code': 'Children Code ', 
            'Combined Pre-Tax Income of HH': 'Pre-Tax Income ', 
            'Education Level Reached by Female HH': 'Female Education Level ', 
            'Education Level Reached by Male HH': 'Male Education Level ', 
            'Family Size': 'Family Size of ', 
            'Female Working Hour Code': 'Female Working Hours ', 
            'Male Working Hour Code': 'Male Working Hours', 
            'Marital Status': 'Marital Status ', 
            'Number of Cats': 'Number of Cats ', 
            'Number of Dogs': 'Number of Dogs ', 
            'Type of Residential Possession': 'Type of Residential Possession '
}

better_demo = pd.concat([pd.get_dummies(demo[col], prefix = prefixes[col]) for index,col in enumerate(demo)], axis=1)

In [261]:
better_demo.head(2)

,Age Female HH _18-24,Age Female HH _25-34,Age Female HH _35-44,Age Female HH _45-54,Age Female HH _55-64,Age Female HH _65 +,Age Female HH _N/A,Age Female HH _No such person,Age Male HH _18-24,Age Male HH _25-34,...,Number of Cats _5,Number of Dogs _0,Number of Dogs _1,Number of Dogs _2,Number of Dogs _3,Number of Dogs _4,Number of Dogs _5,Type of Residential Possession _N/A,Type of Residential Possession _Owner,Type of Residential Possession _Renter
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
